In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time

In [3]:
df = pd.read_csv('/kaggle/input/poems-in-portuguese/portuguese-poems.csv')

In [4]:
text = ' '.join(df['Content'].astype(str))

In [5]:
print(f'Length of text: {len(text)} characters')

Length of text: 12236880 characters


In [6]:
print(text[:250])

Eu não tinha este rosto de hoje,
Assim calmo, assim triste, assim magro,
Nem estes olhos tão vazios,
Nem o lábio amargo.

Eu não tinha estas mãos sem força,
Tão paradas e frias e mortas;
Eu não tinha este coração
Que nem se mostra.

Eu não de


In [7]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

182 unique characters


In [8]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),
    mask_token=None
)

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),
    invert=True,
    mask_token=None
)

In [10]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [11]:
sample_text = text[:50]
sample_text

'Eu não tinha este rosto de hoje,\r\nAssim calmo, ass'

In [12]:
chars = tf.strings.unicode_split(sample_text, input_encoding='UTF-8')
chars

<tf.Tensor: shape=(50,), dtype=string, numpy=
array([b'E', b'u', b' ', b'n', b'\xc3\xa3', b'o', b' ', b't', b'i', b'n',
       b'h', b'a', b' ', b'e', b's', b't', b'e', b' ', b'r', b'o', b's',
       b't', b'o', b' ', b'd', b'e', b' ', b'h', b'o', b'j', b'e', b',',
       b'\r', b'\n', b'A', b's', b's', b'i', b'm', b' ', b'c', b'a', b'l',
       b'm', b'o', b',', b' ', b'a', b's', b's'], dtype=object)>

In [13]:
ids = ids_from_chars(chars)
ids

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([ 41,  89,   4,  82, 139,  83,   4,  88,  77,  82,  76,  69,   4,
        73,  87,  88,  73,   4,  86,  83,  87,  88,  83,   4,  72,  73,
         4,  76,  83,  78,  73,  16,   3,   2,  37,  87,  87,  77,  81,
         4,  71,  69,  80,  81,  83,  16,   4,  69,  87,  87])>

In [14]:
print(text_from_ids(ids).numpy().decode('utf-8'))

Eu não tinha este rosto de hoje,
Assim calmo, ass


# Create training examples and targets

In [15]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(12236880,), dtype=int64, numpy=array([41, 89,  4, ..., 18,  2, 18])>

In [16]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [17]:
seq_length = 100

In [18]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'E' b'u' b' ' b'n' b'\xc3\xa3' b'o' b' ' b't' b'i' b'n' b'h' b'a' b' '
 b'e' b's' b't' b'e' b' ' b'r' b'o' b's' b't' b'o' b' ' b'd' b'e' b' '
 b'h' b'o' b'j' b'e' b',' b'\r' b'\n' b'A' b's' b's' b'i' b'm' b' ' b'c'
 b'a' b'l' b'm' b'o' b',' b' ' b'a' b's' b's' b'i' b'm' b' ' b't' b'r'
 b'i' b's' b't' b'e' b',' b' ' b'a' b's' b's' b'i' b'm' b' ' b'm' b'a'
 b'g' b'r' b'o' b',' b'\r' b'\n' b'N' b'e' b'm' b' ' b'e' b's' b't' b'e'
 b's' b' ' b'o' b'l' b'h' b'o' b's' b' ' b't' b'\xc3\xa3' b'o' b' ' b'v'
 b'a' b'z' b'i' b'o' b's'], shape=(101,), dtype=string)


In [19]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Eu n\xc3\xa3o tinha este rosto de hoje,\r\nAssim calmo, assim triste, assim magro,\r\nNem estes olhos t\xc3\xa3o vazios'
b',\r\nNem o l\xc3\xa1bio amargo.\n\r\nEu n\xc3\xa3o tinha estas m\xc3\xa3os sem for\xc3\xa7a,\r\nT\xc3\xa3o paradas e frias e mortas;\r\nEu n\xc3\xa3o ti'
b'nha este cora\xc3\xa7\xc3\xa3o\r\nQue nem se mostra.\n\r\nEu n\xc3\xa3o dei por esta mudan\xc3\xa7a,\r\nT\xc3\xa3o simples, t\xc3\xa3o certa, t\xc3\xa3o f\xc3\xa1ci'
b'l:\r\n- Em que espelho ficou perdida\r\nA minha face? Para ser grande, s\xc3\xaa inteiro: nada\r\nTeu exagera ou e'
b'xclui.\r\nS\xc3\xaa todo em cada coisa. P\xc3\xb5e quanto \xc3\xa9s\r\nNo m\xc3\xadnimo que fazes.\r\nAssim em cada lago a lua toda\r\nBr'


In [20]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy().decode('utf-8'))

Eu não tinha este rosto de hoje,
Assim calmo, assim triste, assim magro,
Nem estes olhos tão vazios
,
Nem o lábio amargo.

Eu não tinha estas mãos sem força,
Tão paradas e frias e mortas;
Eu não ti
nha este coração
Que nem se mostra.

Eu não dei por esta mudança,
Tão simples, tão certa, tão fáci
l:
- Em que espelho ficou perdida
A minha face? Para ser grande, sê inteiro: nada
Teu exagera ou e
xclui.
Sê todo em cada coisa. Põe quanto és
No mínimo que fazes.
Assim em cada lago a lua toda
Br


In [21]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [22]:
dataset_split = sequences.map(split_input_target)

In [23]:
for input_example, target_example in dataset_split.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Eu n\xc3\xa3o tinha este rosto de hoje,\r\nAssim calmo, assim triste, assim magro,\r\nNem estes olhos t\xc3\xa3o vazio'
Target: b'u n\xc3\xa3o tinha este rosto de hoje,\r\nAssim calmo, assim triste, assim magro,\r\nNem estes olhos t\xc3\xa3o vazios'


In [24]:
for input_example, target_example in dataset_split.take(1):
    print("Input :", text_from_ids(input_example).numpy().decode('utf-8'))
    print("Target:", text_from_ids(target_example).numpy().decode('utf-8'))

Input : Eu não tinha este rosto de hoje,
Assim calmo, assim triste, assim magro,
Nem estes olhos tão vazio
Target: u não tinha este rosto de hoje,
Assim calmo, assim triste, assim magro,
Nem estes olhos tão vazios


# Create training batches

In [25]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset_split
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 100), dtype=tf.int64, name=None), TensorSpec(shape=(None, 100), dtype=tf.int64, name=None))>

# Build the Model

In [30]:
vocab_size = len(ids_from_chars.get_vocabulary())

embedding_dim = 256

rnn_units = 1024

In [31]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.GRU(rnn_units, return_sequences=True),
    tf.keras.layers.Dense(vocab_size)
])



Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [32]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 183) # (batch_size, sequence_length, vocab_size)


In [33]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (64, 100, 256)              │          46,848 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (64, 100, 1024)             │       3,938,304 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (64, 100, 183)              │         187,575 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,172,727 (15.92 MB)

 Trainable params: 4,172,727 (15.92 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [35]:
sampled_indices

array([128, 160, 157, 109,  88,  46,  44,  28,   1,  78,  64,  10,  26,
       182,   0,  46,  85,  17,  70, 104, 170,  88,  71,  25,  34,  39,
        88,  17,  54,  73, 139,  40, 129,   2, 139, 174, 140,  91, 137,
        29, 109, 147, 138, 135,  57,  17,  37,  80, 100,  65, 157,   7,
       119, 170, 124, 135,  54,  22,  50, 134, 128, 170,  78,   6,  62,
        92,  52,  52, 180, 148, 125, 139,  86, 117,  63, 153,   2, 102,
        65,  27,   2,  52,  12, 153,  14, 119, 146, 182,  71,  33, 171,
       113,  62,  53, 135,  78,  24,  26,  42, 173])

Decode these to see the text predicted by this untrained model:

In [37]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy().decode('utf-8'))
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy().decode('utf-8'))

Input:
  teus pés, abro o meu seio 
Procurei fugir de mim, 
Mas sei que sou meu exclusivo fim. 

Sofro, a

Next Char Predictions:
 Òûö°tJH8	j\&6ﬂ[UNK]Jq-b¨―tc5>Ct-ReãDÓ
ã”äwá9°ëâßU-Al¡]ö#Â―ÊßR2NÜÒ―j"ZxPP■ìËãrÀ[ò
¦]7
P(ò*Âêﬂc=‘ºZQßj46F“


# Train the model

In [38]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [39]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 183)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(5.208572, shape=(), dtype=float32)


In [40]:
tf.exp(example_batch_mean_loss).numpy()

182.83278

In [41]:
model.compile(optimizer='adam', loss=loss, metrics=['sparse_categorical_accuracy'])

In [42]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (64, 100, 256)              │          46,848 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (64, 100, 1024)             │       3,938,304 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (64, 100, 183)              │         187,575 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,172,727 (15.92 MB)

 Trainable params: 4,172,727 (15.92 MB)

 Non-trainable params: 0 (0.00 B)

In [43]:
EPOCHS = 20

In [44]:
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/20
1894/1894 ━━━━━━━━━━━━━━━━━━━━ 78s 40ms/step - loss: 2.1130 - sparse_categorical_accuracy: 0.4007
Epoch 2/20
1894/1894 ━━━━━━━━━━━━━━━━━━━━ 75s 39ms/step - loss: 1.4953 - sparse_categorical_accuracy: 0.5430
Epoch 3/20
1894/1894 ━━━━━━━━━━━━━━━━━━━━ 74s 39ms/step - loss: 1.4238 - sparse_categorical_accuracy: 0.5611
Epoch 4/20
1894/1894 ━━━━━━━━━━━━━━━━━━━━ 74s 39ms/step - loss: 1.3877 - sparse_categorical_accuracy: 0.5707
Epoch 5/20
1894/1894 ━━━━━━━━━━━━━━━━━━━━ 75s 39ms/step - loss: 1.3652 - sparse_categorical_accuracy: 0.5767
Epoch 6/20
1894/1894 ━━━━━━━━━━━━━━━━━━━━ 75s 40ms/step - loss: 1.3509 - sparse_categorical_accuracy: 0.5805
Epoch 7/20
1894/1894 ━━━━━━━━━━━━━━━━━━━━ 75s 40ms/step - loss: 1.3427 - sparse_categorical_accuracy: 0.5824
Epoch 8/20
1894/1894 ━━━━━━━━━━━━━━━━━━━━ 76s 40ms/step - loss: 1.3382 - sparse_categorical_accuracy: 0.5835
Epoch 9/20
1894/1894 ━━━━━━━━━━━━━━━━━━━━ 76s 40ms/step - loss: 1.3368 - sparse_categorical_accuracy: 0.5840
Epoch 10/20
1894/18

# Generating text

In [45]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Embedding layer
    x = self.model.layers[0](input_ids)
    # GRU layer
    x = self.model.layers[1](x, initial_state=states)     
    # Get the hidden state of the last timestep
    states = x[:, -1, :]
    # Dense layer
    predicted_logits = self.model.layers[2](x)

    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature

    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [46]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [47]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO: 1984 | pastos bento,s da mir sellenterá 12 São Te alha quel ao fre que valpals Sãd alig anta Blos trios anit fleta à reyenão me seceso Andel inf latira da per not wo kocen an la (Alvos lento. É que lua estrum jes o Iriração mevo olhoredação da o paesme o mumopólio feianda?Noito. Plivestento que me fõglesus ontenha ra peradidado hovrovrido note esestam o olhos se mormões no que dele mas, as vultolhos... Cheirredo de 1965) wingE de que a Lor rega homegando org a seguar-arenterno,
Tuatse to da raciond. Dcome anus gata do 
g..
 Moutyio
O comever-de-tecaverã fo fogesabando mando longindadequ’spore.
O que ne a myis
Alem de fore hulijado
Que voi 
  trien.H de Tantenóna se ororad fel añes quem quis vol añonce empo paixo Se!    Pomentila — o deler dota Bantânte qu'noiraiorioulo à brugrazel,
An The o brania sa nante de Singitos mokt wo lentura nonte pother wockd?
Linentuas falarrído, pess dietatar do mounde,
Renteirescreram de sais, garos delog as: chisto, then thate entre har aídiruid 



In [49]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

results = tf.strings.join(result)
end = time.time()

for text in results:
    print(text.numpy().decode('utf-8'), '\n\n' + '_'*80)

print('\nRun time:', end - start)

ROMEO: Rogrooente e lesceb-Enethe não pálie.D
Lhorrádis | Purdo e and it forençotro cear láien Mo stuandig o comas nãos lhegua dolmá!
Paproe sãe veqdims,des peldadem sabetháse de vid horeedese bunow das o dale ém seyde flta rite. Se eldo saúde águi gorar de luca fuitentrávede em ser palte: Es todo, Láa amas lam poté senhor delfodos vecos dele segaremjas que ao y se para
Cabjoar reifa umaneque cordade eus venco elve no tha and de menó am dilhe tre pedades do solheye bert como hastô com no es nivo me palave,» pagerran esta sietale.
E lar deus de Turisorai e todoureante? Ah!    e não é gretica não começo, sem nãos, ossos iond na dia seentralidõe
 Lernarche dier flemelho
O (Séuxa melna e o fum bomblia deara naia-Talplág onhandowa ouvei; sim on cortresa na curorala ungríuto,
Sim thel insthetw wh hny wer whe talendto inds cam do espout wilenteds noite me cuoldado
escho cos mordi, a caus!f adre glarelentos, abrejo,
come lá virgensas à vivo
       Tode alwmam a dorto à pos cliz olhos os me  



# Export the generator

In [50]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')